# Cognitive, behavioral and social data
**DATASET**: PCL5  
**Author**: Mattia Brocco

MERGE OF DATASETS FOR **R_NEO_PI**
```python
a = pd.read_excel(data_dir + "\\R_NEO_PI_Faked.xlsx")
b = pd.read_excel(data_dir + "\\R_NEO_PI_Honest.xlsx")

a.columns = [" ".join([pd.Series(a.columns).apply(lambda s: np.nan if "Unnamed"
                                                  in s else s).fillna(method = "ffill").tolist()[i],
                       a.loc[0][i]]) for i in range(len(a.columns))]
b.columns = [" ".join([pd.Series(b.columns).apply(lambda s: np.nan if "Unnamed"
                                                  in s else s).fillna(method = "ffill").tolist()[i],
                       b.loc[0][i]]) for i in range(len(b.columns))]

a = a.drop(0).reset_index(drop = True)
b = b.drop(0).reset_index(drop = True)

a["CONDITION"] = "FAKE"
b["CONDITION"] = "HONEST"

pd.concat([a, b], ignore_index = True).to_excel(data_dir + "\\R_NEO_PI.xlsx", index = False)
```

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import support
from engine import Classification

%load_ext autoreload
%autoreload 2

data_dir = ".\\data"

pd.options.display.max_columns = 500

---
## Try to design a pipeline
***

SOURCES  
* [Factor Analysis](https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/factor-analysis/)
* [Likelihood-ratio test](https://en.wikipedia.org/wiki/Likelihood-ratio_test)
* [How can I get statistics to compare nested models in a logistic regression in SPSS?](https://www.ibm.com/support/pages/how-can-i-get-statistics-compare-nested-models-logistic-regression-spss)

In [2]:
### PIPELINE
# Organize datasets
data_collection = {}
for dataset in [f for f in os.listdir(data_dir) if "feather" in f]:
    print(dataset.split(".")[0])
    a, b, c, d = Classification().prepare_data(f"{data_dir}\\{dataset}", "CONDITION")
    e = Classification().variable_selection(a, b, c, d)
    f = Classification().benchmark_models(a, b, c, d, e)
    data_collection[dataset.split(".")[0]] = [a, b, c, d, e, f]
    print(e)
    print(f)
    print("-" * 50)
    print()

BF_df_CTU
5/5 [==============================] - 0s 1ms/step
Sample size: 309
Selected 2 features out of 10
{'Features': [6, 4], 'Wilks test p-value': 1.0, 'Validation passed': False}
Full Logit             0.789474
Logistic Regression    0.781955
SVC                    0.827068
Random Forest          0.827068
Neural Network         0.781955
dtype: float64
--------------------------------------------------

BF_df_OU
5/5 [==============================] - 0s 2ms/step
Sample size: 322
Selected 2 features out of 10
{'Features': [6, 4], 'Wilks test p-value': 1.0, 'Validation passed': False}
Full Logit             0.840580
Logistic Regression    0.782609
SVC                    0.804348
Random Forest          0.804348
Neural Network         0.760870
dtype: float64
--------------------------------------------------

BF_df_V
5/5 [==============================] - 0s 2ms/step
Sample size: 340
Selected 2 features out of 10
{'Features': [4, 7], 'Wilks test p-value': 1.0, 'Validation passed': Fals

In [4]:
a, b, c, d = Classification().prepare_data(".\\data\\PID5_df.feather", "CONDITION")
e = Classification().variable_selection(a, b, c, d)
f = Classification().benchmark_models(a, b, c, d, e)

8/8 [==============================] - 0s 3ms/step
Sample size: 576
Selected 1 features out of 220


In [8]:
e

{'Features': [1], 'Wilks test p-value': 1.0, 'Validation passed': True}

## Instead, use a paper from literature
SOURCES
* [Model-agnostic Feature Importance and Effects with Dependent Features -- A Conditional Subgroup Approach](https://arxiv.org/abs/2006.04628)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance

In [ ]:
X_train, X_test, y_train, y_test = engine.Classification().prepare_data(f"{data_dir}\\BF_df_CTU.csv", "CONDITION")

lr = LogisticRegression(n_jobs = -1, random_state = 42, max_iter = 5e3)
lr.fit(X_train, y_train)

In [ ]:
pi = permutation_importance(lr, X_test, y_test, n_repeats = 30,
                            random_state = 42, scoring = "accuracy")

In [ ]:
plt.bar(x = range(len(pi["importances_mean"])), height = pi["importances_mean"])
plt.scatter(range(len(pi["importances_mean"])),
            pi["importances_mean"] + pi["importances_std"], color = "orange")
plt.scatter(range(len(pi["importances_mean"])),
            pi["importances_mean"] - pi["importances_std"], color = "orange")
plt.show()